<a href="https://colab.research.google.com/github/JacksonXi/optimisation/blob/main/export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import shutil
import sys
import os.path
import numpy as np
import pandas as pd

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))
    
from pyomo.environ import *


     |████████████████████████████████| 8.9 MB 6.7 MB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 160837 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unp

In [2]:
!git clone https://github.com/JacksonXi/optimisation

Cloning into 'optimisation'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 49 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [3]:
cd /content/optimisation/export

/content/optimisation/export


**Generated energy > (non-controlable energy + flexible energy)**

**Obtain consumer ID**

In [4]:
Flexibleindex = pd.read_csv("flexibleIndex.csv")
Consumer1 = list(Flexibleindex['Unnamed: 0'])
Consumer = list(Flexibleindex['Unnamed: 0'])
Consumer

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10']

**Obtain flexibleindex during peak hours** 





**0: ON**


**1: OFF**




In [5]:
flexibleindex = pd.read_csv("flexibleIndex.csv", index_col=0)
flexibleindex

,TV,Heater,WashingMachine,Dryer,Fridger,Airconditioner,InductionCooker
C1,0,0,1,1,0,1,0
C2,1,0,1,0,0,1,0
C3,1,0,0,1,0,1,0
C4,0,1,0,1,1,0,1
C5,1,0,1,0,1,0,0
C6,0,1,0,1,0,1,0
C7,0,0,0,1,0,1,1
C8,1,0,0,1,0,0,1
C9,0,0,0,1,0,0,0
C10,1,1,1,0,0,0,1


**Obtain electrical usage (KWh)**

In [6]:
power = pd.read_csv('power.csv', index_col=0)
power

,TV,Heater,WashingMachine,Dryer,Fridger,Airconditioner,InductionCooker
C1,2,4,4,8,2,5,5
C2,2,7,4,5,1,3,5
C3,2,5,2,5,2,4,4
C4,2,8,3,5,1,4,4
C5,2,5,4,5,2,3,6
C6,1,6,2,6,1,5,5
C7,1,7,4,5,1,5,6
C8,1,7,2,6,1,3,4
C9,1,4,4,6,1,5,5
C10,2,8,2,5,1,4,5


**Obtain power demand (non-controlable loads)**

In [7]:
demand = pd.read_csv('demand.csv', index_col=0)
print(demand)

     demand
C1        3
C2       13
C3        1
C4        1
C5        5
C6        3
C7        4
C8        6
C9        8
C10       5


**Obtain appliance type**

In [8]:
appliances = list(flexibleindex.keys())
appliances

['TV',
 'Heater',
 'WashingMachine',
 'Dryer',
 'Fridger',
 'Airconditioner',
 'InductionCooker']

**Obtain power demand (non-controlable loads and flexible loads)**

In [9]:
for c in Consumer:
    for a in appliances:
        demand['demand'][c] += power[a][c]*flexibleindex[a][c]
demand

,demand
C1,20
C2,22
C3,12
C4,19
C5,13
C6,20
C7,20
C8,17
C9,14
C10,22


**Obtain bid**

In [10]:
Consumer1

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10']

In [11]:
bid = pd.read_csv('bid.csv', index_col=0)
bid

,bid
C1,18
C2,15
C3,19
C4,21
C5,22
C6,20
C7,17
C8,16
C9,18
C10,21


**Sorting the bid**

In [12]:
n = len(bid)
consumer = [] 
while n != 0:
    max = Consumer[0]
    for i in range(0,n):
      if bid['bid'][max] < bid['bid'][Consumer[i]]:
          max = Consumer[i]
      else:
          max = max
    consumer.append(max)      
    Consumer.remove(max)
    n -= 1
print(Consumer)
print(consumer)

[]
['C5', 'C4', 'C10', 'C6', 'C3', 'C1', 'C9', 'C7', 'C8', 'C2']


In [13]:
Rank = list(reversed(range(0, len(consumer))))
# Rank

consumer_rank = pd.DataFrame(list(zip(consumer, Rank)), columns =['Consumer', 'Rank'])
consumer_rank = consumer_rank.set_index('Consumer')
print(consumer_rank)

          Rank
Consumer      
C5           9
C4           8
C10          7
C6           6
C3           5
C1           4
C9           3
C7           2
C8           1
C2           0


In [14]:
for c in consumer:
  consumer_rank['Rank'][c] += 1
consumer_rank

,Rank
Consumer,
C5,10
C4,9
C10,8
C6,7
C3,6
C1,5
C9,4
C7,3
C8,2


**Obtain prosumer ID**

In [15]:
generatedEnergy = pd.read_csv('generated energy.csv')
Prosumer = list(generatedEnergy['Unnamed: 0'])
Prosumer1 = list(generatedEnergy['Unnamed: 0'])
Prosumer

['P1', 'P2', 'P3', 'P4', 'P5', 'P6']

**Obtain generated energy in the community**

In [16]:
generated_energy = pd.read_csv('generated energy.csv', index_col=0)
generated_energy

,generated energy
P1,31
P2,25
P3,39
P4,48
P5,39
P6,42


**Sorting the storage energy**

In [17]:
n = len(generated_energy)
prosumer = [] 
while n != 0:
    max = Prosumer[0]
    for i in range(0,n):
      if generated_energy['generated energy'][max] < generated_energy['generated energy'][Prosumer[i]]:
          max = Prosumer[i]
      else:
          max = max
    prosumer.append(max)      
    Prosumer.remove(max)
    n -= 1
print(prosumer)
print(Prosumer)

['P4', 'P6', 'P3', 'P5', 'P1', 'P2']
[]


In [18]:
Rank = list(reversed(range(0, len(prosumer))))

prosumer_rank = pd.DataFrame(list(zip(prosumer, Rank)), columns =['Prosumer', 'Rank'])
prosumer_rank = prosumer_rank.set_index('Prosumer')
print(prosumer_rank)

          Rank
Prosumer      
P4           5
P6           4
P3           3
P5           2
P1           1
P2           0


In [19]:
for p in prosumer:
  prosumer_rank['Rank'][p] += 1
prosumer_rank

,Rank
Prosumer,
P4,6
P6,5
P3,4
P5,3
P1,2
P2,1


**Energy is allocated in the order of the bid amount**

In [20]:
model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)

model.transport = Var(prosumer, consumer , domain = NonNegativeReals)


# model.revenue = Objective(expr = sum(model.transport[p, c]*bid['bid'][c] for c in consumer for p in prosumer), 
#                            sense = maximize)

model.revenue = Objective(expr = sum(model.transport[p, c]*bid['bid'][c]*(consumer_rank['Rank'][c])*(prosumer_rank['Rank'][p]) for c in consumer for p in prosumer), 
                           sense = maximize)

model.supply = ConstraintList()
for p in prosumer:
      model.supply.add(sum(model.transport[p, c] for c in consumer) <= generated_energy['generated energy'][p])

model.demand = ConstraintList()
for c in consumer:
      model.demand.add(sum(model.transport[p, c] for p in prosumer) <= demand['demand'][c])

results = SolverFactory('cbc').solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 98282.0
  Upper bound: 98282.0
  Number of objectives: 1
  Number of constraints: 17
  Number of variables: 61
  Number of nonzeros: 60
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of crea

In [21]:
model.pprint()

5 Set Declarations
    demand_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    supply_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}
    transport_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain                              : Size : Members
        None :     2 : transport_index_0*transport_index_1 :   60 : {('P4', 'C5'), ('P4', 'C4'), ('P4', 'C10'), ('P4', 'C6'), ('P4', 'C3'), ('P4', 'C1'), ('P4', 'C9'), ('P4', 'C7'), ('P4', 'C8'), ('P4', 'C2'), ('P6', 'C5'), ('P6', 'C4'), ('P6', 'C10'), ('P6', 'C6'), ('P6', 'C3'), ('P6', 'C1'), ('P6', 'C9'), ('P6', 'C7'), ('P6', 'C8'), ('P6', 'C2'), ('P3', 'C5'), ('P3', 'C4'), ('P3', 'C10'), ('P3', 'C6'), ('P3', 'C3'), ('P3', 'C1'), ('P3', 'C9'), ('P3', 'C7'), ('P3', 'C8'), ('P3', 'C2'), ('P5', 'C5'), ('P5', 

In [22]:
for p in prosumer:
    for c in consumer:
        print(p, c, model.transport[p,c]())

P4 C5 13.0
P4 C4 19.0
P4 C10 16.0
P4 C6 0.0
P4 C3 0.0
P4 C1 0.0
P4 C9 0.0
P4 C7 0.0
P4 C8 0.0
P4 C2 0.0
P6 C5 0.0
P6 C4 0.0
P6 C10 6.0
P6 C6 20.0
P6 C3 12.0
P6 C1 4.0
P6 C9 0.0
P6 C7 0.0
P6 C8 0.0
P6 C2 0.0
P3 C5 0.0
P3 C4 0.0
P3 C10 0.0
P3 C6 0.0
P3 C3 0.0
P3 C1 16.0
P3 C9 14.0
P3 C7 9.0
P3 C8 0.0
P3 C2 0.0
P5 C5 0.0
P5 C4 0.0
P5 C10 0.0
P5 C6 0.0
P5 C3 0.0
P5 C1 0.0
P5 C9 0.0
P5 C7 11.0
P5 C8 17.0
P5 C2 11.0
P1 C5 0.0
P1 C4 0.0
P1 C10 0.0
P1 C6 0.0
P1 C3 0.0
P1 C1 0.0
P1 C9 0.0
P1 C7 0.0
P1 C8 0.0
P1 C2 11.0
P2 C5 0.0
P2 C4 0.0
P2 C10 0.0
P2 C6 0.0
P2 C3 0.0
P2 C1 0.0
P2 C9 0.0
P2 C7 0.0
P2 C8 0.0
P2 C2 0.0


In [23]:
if 'ok' == str(results.Solver.status):
    print("Total Community Revenue = ",model.revenue())
    print("\nTransport Table:")
    for p in Prosumer1:
        for c in Consumer1:
            if model.transport[p,c]() > 0:
                print("Transport from ", p," to ", c, ":", model.transport[p,c](),'KWh')
else:
    print("No Valid Solution Found")

Total Community Revenue =  98282.0

Transport Table:
Transport from  P1  to  C2 : 11.0 KWh
Transport from  P3  to  C1 : 16.0 KWh
Transport from  P3  to  C7 : 9.0 KWh
Transport from  P3  to  C9 : 14.0 KWh
Transport from  P4  to  C4 : 19.0 KWh
Transport from  P4  to  C5 : 13.0 KWh
Transport from  P4  to  C10 : 16.0 KWh
Transport from  P5  to  C2 : 11.0 KWh
Transport from  P5  to  C7 : 11.0 KWh
Transport from  P5  to  C8 : 17.0 KWh
Transport from  P6  to  C1 : 4.0 KWh
Transport from  P6  to  C3 : 12.0 KWh
Transport from  P6  to  C6 : 20.0 KWh
Transport from  P6  to  C10 : 6.0 KWh


In [24]:
n = 0
if 'ok' == str(results.Solver.status):
    print("Personal Revenue:\n")
    for c in consumer:
        for p in prosumer:
            if model.transport[p,c]() > 0:
                print("Balance from ", c," to ", p, ":", model.transport[p,c]() * bid['bid'][c])
else:
    print("No Valid Solution Found")

Personal Revenue:

Balance from  C5  to  P4 : 286.0
Balance from  C4  to  P4 : 399.0
Balance from  C10  to  P4 : 336.0
Balance from  C10  to  P6 : 126.0
Balance from  C6  to  P6 : 400.0
Balance from  C3  to  P6 : 228.0
Balance from  C1  to  P6 : 72.0
Balance from  C1  to  P3 : 288.0
Balance from  C9  to  P3 : 252.0
Balance from  C7  to  P3 : 153.0
Balance from  C7  to  P5 : 187.0
Balance from  C8  to  P5 : 272.0
Balance from  C2  to  P5 : 165.0
Balance from  C2  to  P1 : 165.0


In [25]:
n = 0
m = 0
if 'ok' == str(results.Solver.status):
    print("Personal Revenue:\n")
    for p in prosumer:
      for c in consumer:
          n += model.transport[p, c]() * bid['bid'][c] 
          m += model.transport[p, c]() * bid['bid'][c] 
      print("The revenue of",p ,":" , n)
      n = 0
else:
    print("No Valid Solution Found")
print('\nTotal revenue:',m)

Personal Revenue:

The revenue of P4 : 1021.0
The revenue of P6 : 826.0
The revenue of P3 : 693.0
The revenue of P5 : 624.0
The revenue of P1 : 165.0
The revenue of P2 : 0.0

Total revenue: 3329.0


In [26]:
n = 0
if 'ok' == str(results.Solver.status):
    print("The exported electricity to the grid:\n")
    for p in prosumer:
        n = generated_energy['generated energy'][p]
        for c in consumer:
            n -= model.transport[p, c]()
        if n > 0 :
            print(p, "will export ",n , "KWh to the grid") 

The exported electricity to the grid:

P1 will export  20.0 KWh to the grid
P2 will export  25.0 KWh to the grid


In [27]:
n = 0
if 'ok' == str(results.Solver.status):
    print("Personal Revenue:\n")
    for c in Consumer1:
      for p in Prosumer1:
          n += model.transport[p, c]()
      print("The received energy of",c ,":" , n)
      n = 0
else:
    print("No Valid Solution Found")

Personal Revenue:

The received energy of C1 : 20.0
The received energy of C2 : 22.0
The received energy of C3 : 12.0
The received energy of C4 : 19.0
The received energy of C5 : 13.0
The received energy of C6 : 20.0
The received energy of C7 : 20.0
The received energy of C8 : 17.0
The received energy of C9 : 14.0
The received energy of C10 : 22.0
